### this code hopes to allow any cleaving direction 

In [13]:
import numpy as np

import tight_binding_params as tbp

Es = tbp.E['s']
Ep = tbp.E['px']             

# orbital-basis Hamiltonian
H_orb = np.diag([Es, Ep, Ep, Ep])

# change-of-basis matrices
U_orb_to_sp3 = 0.5*np.array([[1, 1, 1, 1],
                             [1, 1,-1,-1],
                             [1,-1, 1,-1],
                             [1,-1,-1, 1]])
U_sp3_to_orb = U_orb_to_sp3.T  

a = (Es + 3*Ep)/4.0
b = (Es -   Ep)/4.0
H_sp3_explicit = np.full((4,4), b)
np.fill_diagonal(H_sp3_explicit, a)

In [14]:

class Atom:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
  

    def __eq__(self, other):
        if isinstance(other, Atom):
            return self.x == other.x and self.y == other.y and self.z == other.z
        return False
    def add(self, delta):
        return Atom(self.x+delta[0],self.y+delta[1],self.z+delta[2] )

    def __hash__(self):
        return hash((self.x, self.y, self.z))

    def __repr__(self):
        return f"Atom({self.x}, {self.y}, {self.z})"
    


In [35]:
import numpy as np

class UnitCell:
    
    @staticmethod
    def normalize(a):
        a = np.asarray(a)
        return a / np.linalg.norm(a)
    @staticmethod
    def _sublattice(atom):    
        return (round(atom.x*4) + round(atom.y*4) + round(atom.z*4)) & 1
    @staticmethod
    def directionalCosine(delta):
        dx,dy,dz = delta
        norm = np.sqrt(dx**2 + dy**2 + dz**2)
        
        if norm != 0:
            l = dx / norm
            m = dy / norm
            n = dz / norm
        return l, m, n
    
    raw_deltas = {
            0: [(+0.25, +0.25, +0.25), (+0.25, -0.25, -0.25),
                (-0.25, +0.25, -0.25), (-0.25, -0.25, +0.25)],
            1: [(-0.25, -0.25, -0.25), (-0.25, +0.25, +0.25),
                (+0.25, -0.25, +0.25), (+0.25, +0.25, -0.25)]
        }
    
    def __init__(self, cleavage_plane = (0,0,1), N= 1):
        self.HKL = cleavage_plane
        self.N = N
        self.R = self.change_of_base_matrix() if self.HKL != (0, 0, 1) else np.eye(3)
        self.ATOM_POSITIONS = [Atom(-.5,.5,0)]
        self.Z_ATOMS = {0: Atom(-.5,.5,0)}
        self.createBasis(0.25,(-0.25,-0.25,0.25))
        self.neighbors, self.danglingBonds = self.mapNeighbors()

    
    def createBasis(self, z, shift):
        shift1 = (-0.25,-0.25,0.25)
        shift2 = (0.25,-.25,0.25)

        
        if z <= self.N:
            newAtom = self.ATOM_POSITIONS[-1].add(shift)
            self.ATOM_POSITIONS.append(newAtom)
            self.Z_ATOMS.update({z : newAtom})
            if shift == shift1:
                self.createBasis(z + .25, shift2)
            else:
                self.createBasis(z + .25, shift1)
    
    def change_of_base_matrix(self):
        new_z = self.normalize(self.HKL)
        global_z = np.array([0, 0, 1])
        if np.allclose(new_z, global_z):
            return np.eye(3)
        new_y = self.normalize(np.cross(global_z, new_z))
        new_x = self.normalize(np.cross(new_y, new_z))
        return np.column_stack([new_x, new_y, new_z])
    
    
    def checkIfAllowed(self, newAtom):
        # check if atoms are in proper cell 
        return not (newAtom.x < 0 or newAtom.y < 0 or newAtom.z < 0 or newAtom.z >= self.N or newAtom.x >= 1 or newAtom.y >= 1)
        
    def checkIfAllowedInZDirection(self, newAtom):
        return not (newAtom.z < 0 or newAtom.z > self.N)
    
    def determine_hybridization(delta):
        # Extract just the signs
        sign_pattern = np.sign(delta)
        
        # Map each sign pattern to its hybridization index
        if np.array_equal(sign_pattern, [1, 1, 1]):       # Type a
            return 0
        elif np.array_equal(sign_pattern, [1, -1, -1]):   # Type b
            return 1
        elif np.array_equal(sign_pattern, [-1, 1, -1]):   # Type c
            return 2
        elif np.array_equal(sign_pattern, [-1, -1, 1]):   # Type d
            return 3
        else:
            Exception("implement this dumbass")
    def mapNeighbors(self):
        """This gives the list of neighbors of an atom
        For example say i am working with atom at 0.25,.25,.25. I find the sublattice and then the neighbors.
        Each neighbors corresponds to an atom in the TB Hamiltonian - via periodicity or direct neighbor
        
        """
        atoms = self.ATOM_POSITIONS
        neighborsMap = {}
        danglingBonds = {}
        for atom in atoms:
            sublattice = UnitCell._sublattice(atom)
            neighborsMap[atom] = []
            danglingBonds[atom] = []
            for delta in UnitCell.raw_deltas[sublattice]:
                neighbor = atom.add(tuple(delta))
                if self.checkIfAllowedInZDirection(neighbor):
                    mappedNeighbor = self.Z_ATOMS[neighbor.z]
                    l,m,n = UnitCell.directionalCosine(delta)
                    neighborsMap[atom].append((mappedNeighbor, delta, l,m,n))
                else:
                    danglingBonds[atom].append( (neighbor, UnitCell.determine_hybridization(delta)))
        
        return neighborsMap, danglingBonds
    
    def printNeighborMap(self):
        for atom in self.neighbors:
            neighbors = [(delta, neighbor) for neighbor, delta, l, m, n in self.neighbors[atom]]
            
            print(f"the atom is {atom} and the neighbors are: {neighbors}")
                    
                
unit_cell = UnitCell(2)
unit_cell.neighbors
unit_cell.printNeighborMap()


ValueError: At least one array has zero dimension

In [33]:
def create_tight_binding(k, N=1, potentialProfile = None):
    kx,ky = k
    
    #print(N)
    unitCell = UnitCell(N = N)
    unitNeighbors = unitCell.neighbors
    danglingBonds = unitCell.danglingBonds
    
    
    numSilicon = len(unitNeighbors.keys())

   
    orbitals = ['s', 'px', 'py', 'pz', 'dxy','dyz','dzx','dx2y2','dz2', 's*']
    numOrbitals = len(orbitals)
    size = numSilicon * numOrbitals 
    A = np.zeros((size, size), dtype=complex)    
    
    atomToIndex = {}
    indexToAtom = {}
    for atom_index,atom in enumerate(unitNeighbors):
        atomToIndex[atom] = atom_index
        indexToAtom[atom_index] = atom
    
    
    # old code with sp3 hybridization 
    for atom_idx, atom in indexToAtom.items():
        hybridizationMatrix = H_sp3_explicit.copy() 
        
        danglingBondsList = danglingBonds[atom]

        for danglingBondAtom, position in danglingBondsList:
            hybridizationMatrix[position,position] += tbp.E['sp3']            
        
        # if there are no dangling bonds this returns the standard diag matrix with onsite energies 
        onsiteMatrix = U_orb_to_sp3 @ hybridizationMatrix @ U_orb_to_sp3.T # go back to orbital basis 

        
        A[atom_idx*10:atom_idx*10 + 4, atom_idx*10:atom_idx*10 + 4] = onsiteMatrix
        A[atom_idx*10 + 4:atom_idx*10 + 9, atom_idx*10 + 4:atom_idx*10 + 9] = np.eye(5) * tbp.E['dxy']
        A[atom_idx*10 + 9,atom_idx*10 + 9] = tbp.E['s*']
  
        # old code with sp3 hybridization 
    for atom_index in range(numSilicon):
        atom = indexToAtom[atom_index]
        neighbors = unitNeighbors[atom]
        for orbitalIndex, orbital in enumerate(orbitals):
            index_i = atom_index * numOrbitals + orbitalIndex
            for neighbor in neighbors:
                atom2, delta, l,m,n = neighbor
                phase = np.exp(2 * np.pi * 1j * (kx*delta[0] + ky*delta[1])) # blochs theorem does not work 
                neighbor_index = atomToIndex[atom2]      
                for secOrbitalIndex, secondOrbital in enumerate(orbitals):
                    index_j = neighbor_index * numOrbitals + secOrbitalIndex

                    hop = tbp.SK[(orbital, secondOrbital)](l, m, n, tbp.V)
                   
                    A[index_i,index_j] += hop * phase   
    
        dagger = lambda A: np.conjugate(A.T)
    if not np.allclose(A, dagger(A)):
        print("H isnt Hermitian")

    eigvals,eigv = np.linalg.eigh(A)
    return eigvals, A
    

[[ 7.84832000e+00+0.j -6.66133815e-16+0.j  1.11022302e-15+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 8.88178420e-16+0.j  1.42292500e+01+0.j -1.00000000e+01+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.00000000e+01+0.j  1.42292500e+01+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   1.37895000e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  1.37895000e+01+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  1.91165000e+01+0.j]]
